In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import os
os.chdir('/content/drive/MyDrive/AI_in_Data_Science/projekt_munka_viragok')

In [3]:
from classok import ImageCaptioner

# IdeficsForVisionText2Text modelt használva

In [ ]:
from transformers import AutoProcessor, IdeficsForVisionText2Text
import torch

model_id = "HuggingFaceM4/idefics-9b-instruct"
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16

processor = AutoProcessor.from_pretrained(model_id)
model = IdeficsForVisionText2Text.from_pretrained(
    model_id,
    torch_dtype=dtype,
    device_map="auto",
    offload_buffers=True
)

# --- Load label map ---
with open("/kaggle/input/pytorch-challange-flower-dataset/cat_to_name.json", "r") as f:
    label_map = json.load(f)

# --- Prompt ---
prompt_template = (
"""User: Provide a detailed description of the flower in the image:

    Example: "The poinsettia has large, vividly colored bracts, often mistaken for petals. The bracts form a star-shaped rosette around small yellow flowers in the center. The bracts come in shades of red, white, pink, orange, or variegated blends, creating a strong contrast with dark green leaves. Leaf texture is smooth or lightly veined, with pointed or rounded tips depending on the cultivar. The flowers are yellow-green and surrounded by tiny red glands that secrete nectar."

    1. What is the overall shape of the flower's bloom? Is it symmetrical or asymmetrical? Describe the petals' shape.
    2. What color are the petals? Are there any gradients or patterns? How does the color change in different light?
    3. Describe the texture of the petals. Are they smooth, velvety, or waxy? Do they reflect light, appearing glossy or matte?
    4. What does the center of the flower look like? Are there visible stamens or pistils? What colors and textures are visible?
    5. What is the size of the flower's bloom? Is it compact or spread out?
    6. How are the petals arranged? Are they spaced evenly, overlapping, or layered? Are there any visible veins or markings?
    7. Are leaves visible? Describe their shape, color, and texture.
    8. What is the condition of the flower? Is it fully open, partially open, or wilting?
    9. Is the flower growing alone or in a group? How are the flowers arranged? Are they at different blooming stages?
    10. Where is the flower found? Is it in a garden, field, or forest? Describe the surroundings.
    11. How does the flower appear under different lighting? Does it have shadows, translucence, or reflections?
    12. Is the flower in full bloom or at a particular stage? How does it compare to other flowers in different blooming stages?
    13. What visual features distinguish this flower, such as shape, color, or texture?
    14. How does the environment affect the flower’s appearance? Is it in rocky, moist, or dry soil?

    Assistant:

    """
)

captioner = ImageCaptioner(
    model=model,
    processor=processor,
    base_folder="/kaggle/input/pytorch-challange-flower-dataset/dataset/train",
    label_map=label_map,
    output_json_path="/kaggle/working/idefics_flower_captions.json",
    prompt=prompt_template,
    model_type="idefics",
    device=device,
    dtype=dtype
)


custom_image_paths = [
    "/kaggle/input/pytorch-challange-flower-dataset/dataset/train/1/image_06734.jpg"]

captioner.caption_selected_images(custom_image_paths)


/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


preprocessor_config.json:   0%|          | 0.00/283 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/500k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.84M [00:00<?, ?B/s]

...

config.json:   0%|          | 0.00/1.41k [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/99.3k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.97G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/7.89G [00:00<?, ?B/s]

# Blip2 modelt használva

In [ ]:
from transformers import Blip2Processor, Blip2ForConditionalGeneration
import torch

model_id = "Salesforce/blip2-flan-t5-xl"
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16  # works best with BLIP-2 on GPU

# Load processor and model for BLIP-2
processor = Blip2Processor.from_pretrained(model_id)
model = Blip2ForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=dtype,
    device_map="auto"
)


# --- Load label map ---
with open("/kaggle/input/pytorch-challange-flower-dataset/cat_to_name.json", "r") as f:
    label_map = json.load(f)

# --- Prompt ---
prompt_template = (
    "Describe the visual appearance of a '{label}' flower in rich detail. "
    "Include petal shape, color, texture, bloom structure, center, leaves, and background."
)




captioner = ImageCaptioner(
    model=model,
    processor=processor,
    base_folder="/kaggle/input/pytorch-challange-flower-dataset/dataset/train",
    label_map=label_map,
    output_json_path="/kaggle/working/blip2_flower_captions.json",
    prompt=prompt_template,
    model_type="blip2",
    device=device,
    dtype=dtype
)


custom_image_paths = [
    "/kaggle/input/pytorch-challange-flower-dataset/dataset/train/1/image_06734.jpg"]

captioner.caption_selected_images(custom_image_paths)

# Llava modelt használva

In [ ]:
from transformers import LlavaProcessor, LlavaForConditionalGeneration
import torch

model_id = "llava-hf/llava-1.5-7b-hf"
device = "cuda" if torch.cuda.is_available() else "cpu"
dtype = torch.float16

processor = LlavaProcessor.from_pretrained(model_id)
model = LlavaForConditionalGeneration.from_pretrained(
    model_id,
    torch_dtype=dtype,
    device_map="auto"
)




# --- Load label map ---
with open("/kaggle/input/pytorch-challange-flower-dataset/cat_to_name.json", "r") as f:
    label_map = json.load(f)

# --- Prompt ---
prompt_template = (
    "<image>\nUSER: Describe the visual appearance of the flower labeled '{label}' in rich detail. "
    "Focus on petal shape, colors, texture, center, leaves, and background.\nASSISTANT:"
)



captioner = ImageCaptioner(
    model=model,
    processor=processor,
    base_folder="/kaggle/input/pytorch-challange-flower-dataset/dataset/train",
    label_map=label_map,
    output_json_path="/kaggle/working/llava_flower_captions.json",
    prompt=prompt_template,
    model_type="llava",
    device=device,
    dtype=dtype
)




custom_image_paths = [
    "/kaggle/input/pytorch-challange-flower-dataset/dataset/train/1/image_06734.jpg"]

captioner.caption_selected_images(custom_image_paths)
